In [1]:
from Crypto.Util.number import getPrime, long_to_bytes, bytes_to_long, inverse, isPrime
from sympy import symbols, gcd, GF

In [2]:
s1 = bytes_to_long(bytes.fromhex("685dba88de1ecf0b4ae5bc84b7ee87f63eb37f697ca9a5ab6af9359341a2fbbf53b9502477cabb1658fdf775a34a0712b04d0fd2679b47ec088e0ab3c0a9a866198077a496bb1de138cd165ca28722dee7c4cc81ac0a3a179095f11981e9c7bcd590576169ed877b5692f42a7d9845bdb7c0bffd4e97541b65321de83e4083c1c8cc93eec59933f42655d7c0ad170ed9a3ea418b582e09a2692fc1965d8372cac678f0dabe1b0cbda93ac9b484feb9d2e96f3ab7e2fc6430da1931281c1870c637866be7fcd69c1b067e001887bb17a57ccd77532ea9dfaa0be1390db5511771dc9e03593e344bf0647ddac395b1fe80a86ad4ea4606fdb8a82fdcf9c846114c"))
s2 = bytes_to_long(bytes.fromhex("356f7e82071f321361075ee85f9b42922662559ed64b253c64ff37b52fe8dcf3ab3163079bc9a12e951f84d2f7a911cbf1b1e8d7cd759a128f21a89b625b07ded33443a2888ca9a455198fd5b4a3fb307f34c704b7dcad88685263f4c3f4cf37f1099f2bd188de72533308c25fc18948dda220e3693b7f3edb689ee489c14e7624932ee8928370c9c1d59b06d1071a259d64c38735b1b586082099919713b669a79e43329f0c20508620982d95b774a57d009540c2ef2835887d229273223272f86fb0b1740937d3fc83d7556ffe634a16fb1faf6125878b06f5d537c21260014e2e67ae47636cbce899c463a3669954253aac3aa89a1c800d3251cf6a36badf"))
c1 = bytes_to_long(bytes.fromhex("0241f53c0690e3faccc3753b6064aef27341b5bef3a10fcbb362251e1f5474a055a04e631af1bb4542351f6051438fc6dbf2011f79cbd85bc667d1097b57818d01d11aa09db0ef221ccf8d9eb16903423702b64a534d49153b49dc47fd5597a96f2a6480d296d36d08ba3438cc193bba6ee2c3ea81ab4dbb029a737c3f5597c8e4b8db8ab06605443eb35160828bc78b1d889814d8811e89efae3d741a481a7bd09483df8ee6d32b56a8d7eb20b275cf3ba5936838da2893f82cbc469f1497f785603e72df1ae1f619e08834588f2e64dd5f4cbbdbc7357dadcd89dbd9e18b0948f9b3f8f6b0df217bd7e8ae5c89a20878ffb127e3cf862baa78cc67ec1012af"))
c2 = bytes_to_long(bytes.fromhex("7499a590fcb19dd0880b77a0dd57f66f6055976100b10053adadaeec18c382c5c3d095b4edd6ee2a5dfdc5790b18ff96e54f093fa62d4b518c1bbe65ad3588a81a1723ce72798ddd06d1eca7be9332a7b754f85582c4c5800d0c778ec320fa53806d122b4f4e436ead12bdf05031d4c181416184932517da985ff503759d128761bd96009c43bf11e45ba60f495235d29a863b7a64d9752868dd9896563fe2cc91df6f092f6d4d7d600b4fbf2b52579a0f2657223a1092c067584aad9997540b25921513f96f2da0c26ffb2ee7578540efc50bc8ab0feeeb24e0e96ebc1e6310dbed880ec5d9788a86bebe72c4b5d9b5c66716e6b84021591372c823c6d78c4e"))
p1 = bytes_to_long(b"Lost modulus had a serious falw in it , we fixed it in this version, This should be secure")
p2 = bytes_to_long(b"If you can't see the modulus you cannot break the rsa , even my primes are 1024 bits , right ?")

$c_1 = p_1^3 [n] \quad c_2 = p_2^3 [n] \quad p_1 = c_1^d [n] \quad p_2 = c_2^d [n]$

$s_1 = f_1^3 [n] \quad s_2 = f_2^3 [n] \quad | f_1 - f_2| \ll n$

In [3]:
def nbits(x: int) -> int:
    return len(bin(x)[2:])

Now we know that $p_1^3 - c_1 \equiv p_2^3 - c_2 \equiv 0 [n]$.

In [4]:
n = int(gcd(p1**3-c1, p2**3-c2))
print(f"n: {nbits(n)} bits")
assert pow(p1, 3, n) == c1
assert pow(p2, 3, n) == c2

n: 2048 bits


Similarly, $s_1 - f_1^3 \equiv s_2 - f_2^3 \equiv 0 [n]$.

From the padding method, we have $f_1 = 2^{128}f + r_1$ and $f_2 = 2^{128}f + r_2$ where $0 \le r_1, r_2 < 2^{128}$.

We use variations of the Coppersmith's attack, namely [Coppersmith’s short-pad attack](https://en.wikipedia.org/wiki/Coppersmith's_attack#Coppersmith%E2%80%99s_short-pad_attack) (using PARI/GP) then [Franklin–Reiter related-message attack](https://en.wikipedia.org/wiki/Coppersmith's_attack#Franklin%E2%80%93Reiter_related-message_attack) (using SymPy).

In [5]:
delta = 294601766759961443379168616135306738383 # from PARI/GP

In [6]:
x = symbols('x')
g1 = ((x+delta)**3 - s1).as_poly(domain=GF(n))
g2 = (x**3 - s2).as_poly(domain=GF(n))
f1 = -gcd(g1, g2).all_coeffs()[-1]
long_to_bytes(f1)[:-16]

b'HTB{Fr4nk1ln_r3t1t3r_sh0rt_p4d_4tt4ck!4nyw4ys_n3v3r_us3_sm0l_3xp_f0r_rs4!1s_th1s_Msg_g01ng_l4rg3r?_0h_y3s_cuz_1_h4v3_t0_Pr3v3nt_Cub3_r00t_4tt4ck}'